# AzureMLChatOnlineEndpoint

>[Azure 机器学习](https://azure.microsoft.com/en-us/products/machine-learning/) 是一个用于构建、训练和部署机器学习模型的平台。用户可以通过模型目录探索要部署的模型类型，该目录提供了来自不同提供商的基础模型和通用模型。
>
>通常，您需要部署模型才能使用其预测（推理）。在 `Azure 机器学习` 中，[在线终结点](https://learn.microsoft.comhttps://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints) 用于部署这些模型并提供实时服务。它们基于 `终结点` 和 `部署` 的概念，允许您将生产工作负载的接口与其提供的实现分离开来。

本笔记本将介绍如何使用托管在 `Azure 机器学习终结点` 上的聊天模型。

In [2]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint

## 设置

您必须[在 Azure ML 上部署模型](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-foundation-models?view=azureml-api-2#deploying-foundation-models-to-endpoints-for-inferencing)或[部署到 Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-open)，并获取以下参数：

* `endpoint_url`: 由终结点提供的 REST 终结点 URL。
* `endpoint_api_type`: 在将模型部署到**专用终结点**（托管的托管基础结构）时，使用 `endpoint_type='dedicated'`。在将模型部署为服务时使用**即用即付**产品（模型即服务）时，使用 `endpoint_type='serverless'`。
* `endpoint_api_key`: 由终结点提供的 API 密钥。

## 内容格式器

`content_formatter` 参数是一个处理程序类，用于转换 AzureML 终结点的请求和响应，以匹配所需模式。由于模型目录中存在范围广泛的模型，每个模型处理数据的方式可能彼此不同，因此提供了一个 `ContentFormatterBase` 类，允许用户按需转换数据。提供以下内容格式器：

* `CustomOpenAIChatContentFormatter`：为遵循 OpenAI API 请求和响应规范的模型（如 LLaMa2-chat）格式化请求和响应数据。

*注意：`langchain.chat_models.azureml_endpoint.LlamaChatContentFormatter` 正在被弃用，并被 `langchain.chat_models.azureml_endpoint.CustomOpenAIChatContentFormatter` 取代。*

您可以实现特定于您模型的自定义内容格式器，该格式器继承自 `langchain_community.llms.azureml_endpoint.ContentFormatterBase` 类。

## 示例

以下各节包含有关如何使用此类的示例：

### 示例：使用实时终结点进行聊天补全

In [7]:
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter,
)
from langchain_core.messages import HumanMessage

chat = AzureMLChatOnlineEndpoint(
    endpoint_url="https://<your-endpoint>.<your_region>.inference.ml.azure.com/score",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="my-api-key",
    content_formatter=CustomOpenAIChatContentFormatter(),
)
response = chat.invoke(
    [HumanMessage(content="Will the Collatz conjecture ever be solved?")]
)
response

AIMessage(content='  The Collatz Conjecture is one of the most famous unsolved problems in mathematics, and it has been the subject of much study and research for many years. While it is impossible to predict with certainty whether the conjecture will ever be solved, there are several reasons why it is considered a challenging and important problem:\n\n1. Simple yet elusive: The Collatz Conjecture is a deceptively simple statement that has proven to be extraordinarily difficult to prove or disprove. Despite its simplicity, the conjecture has eluded some of the brightest minds in mathematics, and it remains one of the most famous open problems in the field.\n2. Wide-ranging implications: The Collatz Conjecture has far-reaching implications for many areas of mathematics, including number theory, algebra, and analysis. A solution to the conjecture could have significant impacts on these fields and potentially lead to new insights and discoveries.\n3. Computational evidence: While the conj

### 示例：按使用付费的聊天补全部署（模型即服务）

In [ ]:
chat = AzureMLChatOnlineEndpoint(
    endpoint_url="https://<your-endpoint>.<your_region>.inference.ml.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="my-api-key",
    content_formatter=CustomOpenAIChatContentFormatter,
)
response = chat.invoke(
    [HumanMessage(content="Will the Collatz conjecture ever be solved?")]
)
response

如果您需要向模型传递其他参数，请使用 `model_kwargs` 参数：

In [ ]:
chat = AzureMLChatOnlineEndpoint(
    endpoint_url="https://<your-endpoint>.<your_region>.inference.ml.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="my-api-key",
    content_formatter=CustomOpenAIChatContentFormatter,
    model_kwargs={"temperature": 0.8},
)

可通过调用时传递参数：

In [ ]:
response = chat.invoke(
    [HumanMessage(content="Will the Collatz conjecture ever be solved?")],
    max_tokens=512,
)
response